# Feature Catalog and Selection Tutorial

This tutorial demonstrates how to configure and select features from **mne-features**, **librosa**, and custom extractors defined in `features/extractors.py`.
We rely on the `config.yaml` file to enable or disable entire groups of features.

## Loading the Configuration
The helper `config.load_config` reads `config.yaml` and validates the structure via pydantic models.

In [ ]:
import pprint
import config
from features import load_feature_catalog

cfg = config.load_config()
pprint.pp(cfg.features.model_dump())

## Building the Catalog
The function `load_feature_catalog` uses the configuration to return a dictionary mapping feature names to extractor callables.

In [ ]:
catalog = load_feature_catalog()
list(catalog.keys())[:10]  # preview

## Selecting Specific Libraries
You can disable an entire block (e.g. `librosa`) by setting `enabled: false` in the `config.yaml` section. When `enable_all` is `false`, only the libraries explicitly enabled will be added.

In [ ]:
cfg.features.librosa.enabled = False
catalog = load_feature_catalog()
sorted(catalog)

## Computing Features
Each extractor takes a 1-D NumPy array (window) and the sampling rate `fs` when required.

In [ ]:
import numpy as np
from unitest.fixtures.synthetic_pd import generate_synthetic_partial_discharge

sample = generate_synthetic_partial_discharge(num_good=1, num_fault=0, length=256)
window = sample.iloc[0, :-1].to_numpy(float)
line_length = catalog['line_length'](window, fs=256.0)
line_length

## Batch Extraction from Cleaned Data
The helper function `features.extract_from_clean()` scans `root_dir` for cleaned windows and saves features under `2_feature_engineering/` grouped by theme and cleaning method.

In [ ]:
from features import extract_from_clean

# This will write Parquet feature files into outputs/features/*
extract_from_clean(fs=256.0)